In [ ]:
# !!! Need to pip install many of these !!!
import numpy as np
import pythoncom
from dcam import *
import cv2
from screeninfo import get_monitors
import signal
import os
from astropy.io import fits
import time
import threading
import matplotlib.pyplot as plt
import serial
import pandas as pd
import sys
from FWxC_COMMAND_LIB import (
    FWxCListDevices,
    FWxCOpen,
    FWxCGetPosition,
    FWxCSetPosition,
    FWxCClose,
    FWxCGetPositionCount
)

In [ ]:
"""filter wheel controls, copied"""
def list_devices():
    # Retrieves and returns a list of available filter wheel device IDs.
    devices = FWxCListDevices()
    try:
        if isinstance(devices, bytes):
            txt = devices.decode('utf-8').strip()
            dev_list = [d for d in txt.replace(',', ' ').split() if d]
        else:
            dev_list = [dev[0] for dev in devices]
    except Exception:
        dev_list = [dev[0] for dev in devices]
    return dev_list
def open_device(device_id, nBaud=115200, timeout=3):
    if isinstance(device_id, bytes):
        serial_str = device_id.decode('utf-8')
    else:
        serial_str = device_id
    handle = FWxCOpen(serial_str, nBaud, timeout)
    if handle < 0:
        raise RuntimeError(f"Failed to open device {serial_str} (error code {handle})")
    return handle
def get_position(handle):
    # Gets the current position of the filter wheel.
    pos = [0]
    result = FWxCGetPosition(handle, pos)
    if result < 0:
        raise RuntimeError(f"GetPosition failed with error code {result}")
    return pos[0]
def get_position_count(handle):
    # Returns the total number of positions on the wheel.
    count = [0]
    result = FWxCGetPositionCount(handle, count)
    if result < 0:
        raise RuntimeError(f"GetPositionCount failed (error code {result})")
    return count[0]
def set_position(handle, position):
    # Sets the filter wheel to the given position (integer index).
    result = FWxCSetPosition(handle, int(position))
    if result < 0:
        raise RuntimeError(f"SetPosition({position}) failed with error code {result}")
def close_device(handle):
    # Closes the connection to the filter wheel.
    FWxCClose(handle)
def init_wheels(print_stuff=False):
    if print_stuff: print("Finding filter wheel devices...")
    devices = list_devices()
    if not devices:
        print("No filter wheel devices found.")
        sys.exit(1)
    if print_stuff:
        print("Found devices:")
        for idx, dev in enumerate(devices):
            print(f" [{idx}] {dev}")
    handle_6 = None
    counter = 0
    while not handle_6:
        # init filter wheels to no ND and 640nm
        for i in range(len(devices)):
            handle = open_device(devices[i])
            max_pos = get_position_count(handle)
            if print_stuff: print(max_pos)
            if max_pos > 1:
                if max_pos == 6:
                    set_position(handle, 1)
                    wheel6_port = i
                    handle_6 = handle
                elif max_pos == 12:
                    set_position(handle, 7) #red light
                    wheel12_port = i
                    handle_12 = handle
            else:
                close_device(handle)
        counter += 1
        if counter > 2:
            break
    # return handle_6, handle_12, wheel6_port, wheel12_port
    return handle_6
# close connection to wheels
def close_wheels():
    devices = list_devices()
    for i in range(len(devices)):
        handle = open_device(devices[i])
        close_device(handle)

In [ ]:
init_wheels(print_stuff=True)
close_wheels()

In [ ]:
"""functions for picoammeter, copied"""
def send_cmd(ser, cmd):
    """Send a command and wait a bit"""
    ser.write((cmd + '\r').encode())
    time.sleep(0.1)

def query(ser, cmd):
    """Send a command and read the response"""
    send_cmd(ser, cmd)
    return ser.readline().decode().strip()

def init_cmds(ser):

    # Reset and configure instrument
    print(query(ser, '*IDN?'))
    send_cmd(ser, '*RST')
    send_cmd(ser, ':FORM:ELEM READ')
    send_cmd(ser, 'TRIG:DEL 0')
    send_cmd(ser, 'TRIG:COUNT 1')
    send_cmd(ser, 'SENS:CURR:NPLC 6')
    send_cmd(ser, 'SENS:CURR:RANG 0.000002')
    send_cmd(ser, 'SENS:CURR:RANG:AUTO ON')
    send_cmd(ser, 'SYST:AZER:STAT OFF')
    send_cmd(ser, 'DISP:ENAB ON')
    send_cmd(ser, ':SYST:ZCH:STAT OFF')

    # Wait for settings to take effect
    time.sleep(0.5)

In [ ]:
"""run picoammeter, copied"""
ser = serial.Serial(
    port='COM5',         # Replace with your actual port
    baudrate=9600,
    bytesize=serial.EIGHTBITS,
    parity=serial.PARITY_NONE,
    stopbits=serial.STOPBITS_ONE,
    timeout=2,
    xonxoff=False,
    rtscts=False,
    dsrdtr=False
)
init_cmds(ser)


#417 reads/min with no time.sleep
read_time = .2 # time in minutes to read for
num_of_reads = round(417*read_time)
readings = np.zeros(num_of_reads)

# Trigger a read and fetch result
for i in range(num_of_reads):
    time.sleep(0.001)
    current = query(ser, 'READ?')
    current = float(current)
    if current < 0:
        current *= -1
    readings[i] = current

ser.close()
plt.scatter(range(num_of_reads), readings)
plt.show()


In [ ]:
def take_ramp_hamamatsu(
    iDevice,
    exp_low,
    exp_high,
    num_exp=10,
    num_frames=1,
    img_shape=(1000,1000),
    save_path=None,
    full=False,
    cam=None,
    big_file=True,
):
    import os
    import time
    import pythoncom
    import numpy as np
    from astropy.io import fits
    from dcam import Dcam, Dcamapi, DCAM_IDPROP, DCAM_IDSTR
    """Take a ramp of exposures; stream to disk if big_file=True."""
    pythoncom.CoInitializeEx(pythoncom.COINIT_MULTITHREADED)
    if not Dcamapi.init():
        raise RuntimeError(f"DCAM‑API init failed: {Dcamapi.lasterr()}")
    try:
        cam = Dcam(iDevice)
        if not cam.dev_open():
            raise RuntimeError(f"dev_open failed: {cam.lasterr()}")

        full_h = int(cam.prop_getvalue(DCAM_IDPROP.IMAGE_HEIGHT))
        full_w = int(cam.prop_getvalue(DCAM_IDPROP.IMAGE_WIDTH))
        ty, tx = (full_h, full_w) if full else img_shape
        H, W = ty, tx

        # set subarray to full sensor if desired
                # Configure subarray: full sensor or centered crop
        cam.prop_setvalue(DCAM_IDPROP.SUBARRAYMODE, 1)
        if full:
            hp = 0
            vp = 0
            wsize = full_w
            hsize = full_h
        else:
            # Center subarray of size (tx, ty)
            hp = (full_w - tx) // 2  # horizontal offset
            vp = (full_h - ty) // 2  # vertical offset
            wsize = tx
            hsize = ty
        cam.prop_setvalue(DCAM_IDPROP.SUBARRAYHPOS, hp)
        cam.prop_setvalue(DCAM_IDPROP.SUBARRAYVPOS, vp)
        cam.prop_setvalue(DCAM_IDPROP.SUBARRAYHSIZE, wsize)
        cam.prop_setvalue(DCAM_IDPROP.SUBARRAYVSIZE, hsize)
        dprop = DCAM_IDPROP.READOUTSPEED
        cam.prop_setvalue(dprop, 1)
        print("Set", 'SOMEPROP', "→", cam.prop_getvalue(dprop))

        # Allocate buffer for one subarray frame
        cam.buf_alloc(1)
        exp_times = np.logspace(np.log10(exp_low), np.log10(exp_high), num_exp)

        # conditional storage
        if not big_file:
            img_array = np.zeros((num_exp, num_frames, ty, tx), dtype=np.uint16)
        else:
            img_array = np.zeros((num_exp, num_frames, ty, tx), dtype=np.uint16)
        img_times = np.zeros((num_exp, num_frames))

                # main loop
        if big_file:
            for i, t in enumerate(exp_times):
                print(f"Taking frames at {t:.5f}s ({i+1}/{num_exp})")
                cam.prop_setvalue(DCAM_IDPROP.EXPOSURETIME, float(t))

                # collect this exposure's frames
                if big_file:
                    # prepare on-disk memmap for this exposure
                    cube_dat = os.path.join(save_path, f"ramp_exp_{i+1:02d}.dat")
                    cube_mem = np.memmap(cube_dat, dtype=np.uint16, mode='w+', shape=(num_frames, H, W))
                exp_stack = [] if not big_file else None
                for j in range(num_frames):
                    if not cam.cap_snapshot():
                        raise RuntimeError(f"cap_snapshot failed: {cam.lasterr()}")
                    timeout_ms = int(t*1000) + 2000
                    while True:
                        if cam.wait_capevent_frameready(timeout_ms): break
                        if cam.lasterr().is_timeout(): continue
                        raise RuntimeError(f"wait_frameready failed: {cam.lasterr()}")
                    data = cam.buf_getlastframedata()
                    frame = data.astype(np.uint16)
                    if not full:
                        sy = (data.shape[0] - ty)//2; sx = (data.shape[1] - tx)//2
                        frame = frame[sy:sy+ty, sx:sx+tx]
                    img_times[i, j] = time.time()
                    if big_file:
                        # write directly to memmap slice
                        cube_mem[j] = frame
                    else:
                        img_array[i, j] = frame

            if not big_file:
                for i, t in enumerate(exp_times):
                    # set the exposure time property (seconds)
                    print(f"takings frames at {t:.5f} ({i+1}/{num_exp})")
                    cam.prop_setvalue(DCAM_IDPROP.EXPOSURETIME, float(t))
                    for j in range(num_frames):
                        if not cam.cap_snapshot():
                            raise RuntimeError(f"cap_snapshot failed: {cam.lasterr()}")
                        timeout_ms = int(t*1000) + 2000
                        while True:
                            if cam.wait_capevent_frameready(timeout_ms):
                                break
                            if cam.lasterr().is_timeout():
                                continue
                            raise RuntimeError(f"wait_frameready failed: {cam.lasterr()}")
                        # pull out the image
                        data = cam.buf_getlastframedata()            # should be 2D numpy
                        if data.ndim != 2:
                            # fallback reshape if needed
                            data = data.reshape(full_h, full_w)


                        if full:
                            crop = data.astype(np.uint16)
                        else:
                            sy = (data.shape[0] - ty)//2
                            sx = (data.shape[1] - tx)//2
                            crop = data[sy:sy+ty, sx:sx+tx].astype(np.uint16)
                        img_array[i,j] = crop
                        img_times[i,j] = time.time()
            # write this exposure's stack to FITS
            if save_path:
                if big_file:
                    os.makedirs(save_path, exist_ok=True)
                    cube_path = os.path.join(save_path, f"ramp_exp_{i+1:02d}.fits")
                    if not big_file:
                        cube = img_array[i]
                    else:
                        # flush memmap and read from disk for FITS
                        cube_mem.flush()
                        cube = np.memmap(cube_dat, dtype=np.uint16, mode='r', shape=(num_frames, H, W))
                    hdu = fits.PrimaryHDU(cube)
                    # headers
                    hdr = hdu.header
                    hdr['DATE']     = time.strftime('%Y-%m-%dT%H:%M:%S', time.gmtime())
                    hdr['DATE-OBS'] = time.strftime('%Y-%m-%dT%H:%M:%S', time.gmtime())
                    hdr['EXPTIME']  = cam.prop_getvalue(DCAM_IDPROP.EXPOSURETIME)
                    hdr['TEMP']     = cam.prop_getvalue(DCAM_IDPROP.SENSORTEMPERATURE)
                    hdr['MODE']     = int(cam.prop_getvalue(DCAM_IDPROP.READOUTSPEED))
                    hdr['GAIN']     = cam.prop_getvalue(DCAM_IDPROP.CONTRASTGAIN)
                    hdr['SENSNAME'] = cam.dev_getstring(DCAM_IDSTR.MODEL)
                    hdu.writeto(cube_path, overwrite=True)
                    print(f"Saved {cube_path}")
                else:
                     if save_path:
                        os.makedirs(save_path, exist_ok=True)
                        cube = img_array[i]  # shape (num_frames, ty, tx)
                        hdu = fits.PrimaryHDU(cube)
                        hdr = hdu.header
                        # headers
                        hdr['DATE']    = time.strftime('%Y-%m-%dT%H:%M:%S', time.gmtime())
                        hdr['DATE-OBS']    = time.strftime('%Y-%m-%dT%H:%M:%S', time.gmtime())
                        hdr['EXPTIME'] = cam.prop_getvalue(DCAM_IDPROP.EXPOSURETIME)
                        hdr['TEMP']    = cam.prop_getvalue(DCAM_IDPROP.SENSORTEMPERATURE)
                        hdr['MODE']    = int(cam.prop_getvalue(DCAM_IDPROP.READOUTSPEED))
                        hdr['GAIN']    = cam.prop_getvalue(DCAM_IDPROP.CONTRASTGAIN)
                        hdr['SENSNAME']= cam.dev_getstring(DCAM_IDSTR.MODEL)
                        fname = os.path.join(save_path, f"ramp_exp_{i+1:02d}.fits")
                        hdu.writeto(fname, overwrite=True)

        cam.buf_release()
        cam.dev_close()
        cam.dev_close()

    finally:
        Dcamapi.uninit()
        pythoncom.CoUninitialize()

    return exp_times, img_array, img_times.flatten()


In [ ]:
"""gain calculator"""
def capture_frame_hama(dcam, exp, timeout_ms=None):
    dcam.prop_setvalue(DCAM_IDPROP.EXPOSURETIME, float(exp))
    #set mode to ultra quiet
    # print("readoutspeed: ", int(dcam.prop_getvalue(DCAM_IDPROP.READOUTSPEED)))
    if not dcam.cap_snapshot():
        raise RuntimeError(f"cap_snapshot failed: {dcam.lasterr()}")
    if timeout_ms is None:
        timeout_ms = int(exp*1000) + 2000
    while True:
        if dcam.wait_capevent_frameready(timeout_ms):
            break
        if dcam.lasterr().is_timeout():
            continue
        raise RuntimeError(f"wait_frameready error: {dcam.lasterr()}")
    data = dcam.buf_getlastframedata()
    # Ensure it's 2D; if flattened, reshape
    if data.ndim == 1:
        h = int(dcam.prop_getvalue(DCAM_IDPROP.IMAGE_HEIGHT))
        w = int(dcam.prop_getvalue(DCAM_IDPROP.IMAGE_WIDTH))
        data = data.reshape(h, w)
    return data.astype(float)

def measure_ptc_point_hama(dcam, exp, n_frames=10, print_stuff=False):
    if print_stuff:
        print(f"  Capturing {n_frames} flats at {exp}s…")
    flats = np.stack([capture_frame_hama(dcam, exp)
                      for _ in range(n_frames)])
    mu_flat = flats.mean()
    var_flat= flats.var(ddof=1)

    # prompt user to close shutter for darks
    cmd = ''
    while cmd.lower() != 'y':
        cmd = input("Close shutter and press y: ")

    if print_stuff:
        print(f"  Capturing {n_frames} darks at {exp}s…")
    darks = np.stack([capture_frame_hama(dcam, exp)
                      for _ in range(n_frames)])
    mu_dark = darks.mean()
    var_dark= darks.var(ddof=1)

    # prompt to reopen shutter
    cmd = ''
    while cmd.lower() != 'y':
        cmd = input("Reopen shutter and press y: ")

    mu = mu_flat - mu_dark
    var_diff = var_flat - var_dark
    if var_diff <= 0:
        raise ValueError(f"Non‐positive variance at {exp}s: {var_diff}")
    return mu, var_diff

def get_gain_hama(iDevice=0, exp_list=[0.001, 0.01,0.1,1], n_frames=10, print_stuff=False):
    pythoncom.CoInitializeEx(pythoncom.COINIT_MULTITHREADED)
    if not Dcamapi.init():
        raise RuntimeError(f"DCAM‑API init failed: {Dcamapi.lasterr()}")
    try:
        dcam = Dcam(iDevice)
        if not dcam.dev_open():
            raise RuntimeError(f"dev_open failed: {dcam.lasterr()}")
        # Allocate single‐frame buffer
        # if not dcam.buf_alloc(1):
        #     raise RuntimeError(f"buf_alloc failed: {dcam.lasterr()}")

        full_h = int(dcam.prop_getvalue(DCAM_IDPROP.IMAGE_HEIGHT))
        full_w = int(dcam.prop_getvalue(DCAM_IDPROP.IMAGE_WIDTH))
        dcam.prop_setvalue(DCAM_IDPROP.SUBARRAYMODE, 1)
        print("submode", dcam.prop_getvalue(DCAM_IDPROP.SUBARRAYMODE))
        dcam.prop_setvalue(DCAM_IDPROP.SUBARRAYHPOS,  0)
        dcam.prop_setvalue(DCAM_IDPROP.SUBARRAYVPOS,  0)
        dcam.prop_setvalue(DCAM_IDPROP.SUBARRAYHSIZE, full_w)
        dcam.prop_setvalue(DCAM_IDPROP.SUBARRAYVSIZE, full_h)

        dprop = DCAM_IDPROP.READOUTSPEED
        dcam.prop_setvalue(dprop, 1)
        print("Set", 'SOMEPROP', "→", dcam.prop_getvalue(dprop))

        dcam.buf_alloc(1)

        data = []
        for t in exp_list:
            if print_stuff:
                print(f"Measuring gain point at {t:.3f}s")
            mu, var_diff = measure_ptc_point_hama(dcam, t, n_frames, print_stuff)
            data.append((mu, var_diff))
            if print_stuff:
                print(f"  μ={mu:.1f}, var_diff={var_diff:.1f}")
        # Cleanup camera
        dcam.buf_release()
        dcam.dev_close()

    finally:
        Dcamapi.uninit()
        pythoncom.CoUninitialize()

    # Fit σ² = (1/g) μ
    mus, vars_ = zip(*data)
    mus  = np.array(mus)
    vars_ = np.array(vars_)
    slope, = np.linalg.lstsq(mus.reshape(-1,1), vars_, rcond=None)[0:1]
    gain = 1.0 / slope

    if print_stuff:
        print(f"Measured gain: {gain[0]:.3f} e⁻/ADU")

    return gain[0], {'exp':exp_list, 'mu':mus, 'var_diff':vars_}


In [ ]:
e_per_adu, stats = get_gain_hama(
    iDevice=0,
    exp_list=[0.0001, 0.001, 0.01, 0.1, 0.2, 0.5, 1.0],
    n_frames=5,
    print_stuff=True
)

In [ ]:
"""var exp"""
def picoam_worker_exp(ser, stop_event, readings_list, times_list, sample_interval=0.004):
    while not stop_event.is_set():
        time.sleep(sample_interval)
        raw = query(ser, 'READ?')
        readings_list.append(float(raw))
        times_list.append(time.time())
    ser.close()

def camera_with_picoam(
    ser,
    iDevice,
    exp_low,
    exp_high,
    num_exp=10,
    num_frames=1,
    img_shape=(1000,1000),
    save_path=None,
    bit_depth=16,
    full = False,
    cam=None
):
    # Start picoammeter thread
    stop_evt = threading.Event()
    picoam_readings = []
    picoam_times = []
    t = threading.Thread(
        target=picoam_worker_exp,
        args=(ser, stop_evt, picoam_readings, picoam_times)
    )
    t.start()

    # Run the Hamamatsu ramp
    exp_times, img_array, img_times = take_ramp_hamamatsu(
        iDevice=iDevice,
        exp_low=exp_low,
        exp_high=exp_high,
        num_exp=num_exp,
        num_frames=num_frames,
        img_shape=img_shape,
        save_path=save_path,
        full=full,
        cam=cam
    )

    # Stop picoammeter
    stop_evt.set()
    t.join()

    # Interpolate currents onto the image timestamps
    picoam_readings = np.array(picoam_readings)
    picoam_times    = np.array(picoam_times)
    matched_readings = np.interp(img_times, picoam_times, picoam_readings)

    # Flatten exposures to per-frame
    exp_flat = np.repeat(exp_times, num_frames)
    return exp_flat, matched_readings, img_array, img_times

def collect_var_exp(
    iDevice=0,
    com_port='COM5',
    bit_depth=16,
    num_exp=10,
    num_frames=5,
    exp_low=0.001,
    exp_high=0.5,
    img_shape=(1000,1000),
    save_path=None,
    full=False,
    cam=None
):

    # Open & init picoammeter
    ser = serial.Serial(
        port=com_port, baudrate=9600, timeout=2,
        bytesize=serial.EIGHTBITS,
        parity=serial.PARITY_NONE,
        stopbits=serial.STOPBITS_ONE
    )
    init_cmds(ser)

    exp_flat, currents, exp_img_array, img_times = camera_with_picoam(
        ser=ser,
        iDevice=iDevice,
        exp_low=exp_low,
        exp_high=exp_high,
        num_exp=num_exp,
        num_frames=num_frames,
        img_shape=img_shape,
        save_path=save_path,
        bit_depth=bit_depth,
        full=full,
        cam=cam
    )

    exp_times = exp_flat.reshape(num_exp, num_frames).mean(axis=1)
    # Compute per-exposure means
    I_flat = currents.reshape(-1, num_frames).mean(axis=1)
    Q_ramp = I_flat * exp_times

    per_frame    = exp_img_array.mean(axis=(2,3))
    mean_signal  = per_frame.mean(axis=1)
    std_signal   = per_frame.std(axis=1)

    exp_ms = exp_times * 1e3
    bit_max = 2**bit_depth - 1
    threshold = 0.5 * bit_max
    exp_50 = (np.interp(threshold, mean_signal, exp_ms))/1000

    # save linearity data
    if save_path:
        df = pd.DataFrame({
            'exposure_ms': exp_ms,
            'mean_ADU':    mean_signal,
            'std_ADU':     std_signal
        })
        csv_path = f"{save_path}/exp_{exp_low}-{exp_high}_linearity.csv"
        df.to_csv(csv_path, index=False)
        print(f"Data saved to {csv_path}")

    return I_flat, Q_ramp, mean_signal, std_signal, bit_depth, exp_times, exp_low, exp_high, exp_50

# copied
def var_exp_graphs(
    mean_signal, std_signal, bit_depth,
    exp_times, exp_low, exp_high,
    save=False, e_per_adu=None,
    min_percent=0.05, max_percent=0.95,
    percent_change=False,
    zoom=False
):
    exp_ms = exp_times * 1e3
    bit_max = 2**bit_depth - 1

    # Find linear region
    low_idx = np.argmax(mean_signal >= min_percent*bit_max)
    high_idx = np.argmax(mean_signal >= max_percent*bit_max)
    x_percent = exp_ms[low_idx]

    if percent_change:
        # for finding % change in exp time between readings
        delta_exp_ms    = np.diff(exp_ms)           # length num_exp-1
        delta_signal    = np.diff(mean_signal)      # length num_exp-1
        percent_change  = delta_signal / mean_signal[:-1] * 100
        exp_mid_ms      = (exp_ms[:-1] + exp_ms[1:]) / 2
        plt.figure()
        plt.plot(exp_mid_ms, percent_change, 'o-')
        plt.xscale('log')
        plt.xlabel('Exposure (ms) [mid‐point]')
        plt.ylabel('Percent Δ signal (%)')
        plt.title('Percent Change in Signal Between Exposures')
        plt.grid(True, ls='--', lw=0.5)
        plt.tight_layout()
        plt.show()

    # fit
    m, b = np.polyfit(
        exp_ms[low_idx : high_idx + 1],
        mean_signal[low_idx : high_idx + 1],
        1
    )

    if zoom:
        fig, (ax1, ax2, ax3) = plt.subplots(1,3, figsize=(12,4))
    else:
        fig, (ax1, ax2) = plt.subplots(1,2, figsize=(12,4))

    ax1.errorbar(exp_ms, mean_signal, yerr=std_signal,
                 fmt='o', capsize=3, label='Measured')
    # plot fit line over linear regime
    x_lin = np.linspace(exp_ms[0], exp_ms[high_idx], 200)
    ax1.plot(x_lin, m*x_lin + b, 'r--', label=f'Linear fit: y={m:.3f}*x+{b:.1f}(ADU)')
    # ideal line
    ax1.plot(x_lin, m*x_lin, 'b-.', label=f'Ideal: y={m:.3f}*x')
    # markers
    ax1.axhline(bit_max,     ls=':', color='gray')
    ax1.text(0.99, bit_max, 'Saturation', ha='right', va='bottom', transform=ax1.get_yaxis_transform())
    ax1.axvline(x_percent,         ls=':', color='gray')
    ax1.text(x_percent, 0.01, f'{min_percent*100:.1f}%', va='bottom', ha='center', transform=ax1.get_xaxis_transform())
    ax1.axvline(exp_ms[high_idx], ls=':', color='gray')
    ax1.text(exp_ms[high_idx], 0.01, f'{max_percent*100:.0f}%', va='bottom', ha='center', transform=ax1.get_xaxis_transform())
    # ax1.set_xscale('log')
    ax1.set_xlabel('Exposure (ms)')
    ax1.set_ylabel('Mean signal (ADU)')
    ax1.set_title('a) Full Response')
    ax1.legend(fontsize='small')
    ax4 = ax1.twinx()
    low, high = ax1.get_ylim()
    ax4.set_ylim(low*e_per_adu, high*e_per_adu)
    ax4.set_ylabel('Mean signal (e-)')

    if zoom:
        # for zoom in on 0–x_percent
        mask_percent = exp_ms <= x_percent
        ax3.errorbar(exp_ms[mask_percent], mean_signal[mask_percent],
                     yerr=std_signal[mask_percent],
                     fmt='o', capsize=3)
        # show the linear‐fit there too (through origin)
        x_zoom = np.linspace(0, x_percent, 100)
        ax3.plot(x_zoom, m*x_zoom + b, 'r--')
        ax3.plot(x_zoom, m*x_zoom, 'b-.')
        ax3.set_xlim(0, x_percent*1.05)  # a bit of padding
        ax3.set_xscale('linear')
        ax3.set_yscale('linear')
        ax3.set_xlabel('Exposure (ms)')
        ax3.set_title(f'b) 0-{min_percent*100:.0f}% Region')
        ax6 = ax3.twinx()
        low, high = ax3.get_ylim()
        ax6.set_ylim(low*e_per_adu, high*e_per_adu)
        ax6.set_ylabel('Mean signal (e-)')
        ax6.set_yscale('linear')

    # left: full range
    ax2.errorbar(exp_ms, mean_signal, yerr=std_signal,
                 fmt='o', capsize=3, label='Measured')
    # plot fit line over linear regime
    x_lin = np.linspace(exp_ms[0], exp_ms[high_idx], 200)
    ax2.plot(x_lin, m*x_lin + b, 'r--', label=f'Linear fit: y={m:.3f}*x+{b:.1f}(ADU)')
    # ideal line
    ax2.plot(x_lin, m*x_lin, 'b-.', label=f'Ideal: y={m:.3f}*x')
    # markers
    ax2.axhline(bit_max,     ls=':', color='gray')
    ax2.text(0.99, bit_max, 'Saturation', ha='right', va='bottom', transform=ax2.get_yaxis_transform())
    ax2.axvline(x_percent,         ls=':', color='gray')
    ax2.text(x_percent, 0.01, f'{min_percent*100:.0f}%', va='bottom', ha='center', transform=ax2.get_xaxis_transform())
    ax2.axvline(exp_ms[high_idx], ls=':', color='gray')
    ax2.text(exp_ms[high_idx], 0.01, f'{max_percent*100:.0f}%', va='bottom', ha='center', transform=ax2.get_xaxis_transform())
    ax2.set_xscale('log')
    ax2.set_yscale('log')
    ax2.set_xlabel('Exposure (ms)')
    ax2.set_ylabel('Mean signal (ADU)')
    ax2.set_title('b) Full Response, log scale')
    ax5 = ax2.twinx()
    low, high = ax2.get_ylim()
    ax5.set_ylim(low*e_per_adu, high*e_per_adu)
    ax5.set_ylabel('Mean signal (e-)')
    ax5.set_yscale('log')

    ax2.grid(True, ls='--', lw=0.5)
    ax1.grid(True, ls='--', lw=0.5)
    if zoom: ax3.grid(True, ls='--', lw=0.5)
    plt.tight_layout()
    plt.show()
    if save:
        fig.savefig(f"exp_{exp_low}-{exp_high}_graphs.png", dpi=300)
        print("Figure saved")
    plt.show()

In [ ]:
# Collect data
I_flat, Q_ramp, mean_signal, std_signal, bd, exps, lo, hi, exp_50 = collect_var_exp(
    com_port='COM5',
    num_exp=80,
    num_frames=25,
    exp_low=.0001,
    exp_high=10.0,
    full=True,
    save_path="C:/Users/Jonah/fits_imgs",

)
#
# Plot
# var_exp_graphs(
#     mean_signal=mean_signal,
#     std_signal=std_signal,
#     bit_depth=bd,
#     exp_times=exps,
#     exp_low=lo,
#     exp_high=hi,
#     min_percent=0.005,
#     e_per_adu=e_per_adu,
#     percent_change=False,
#     zoom=False,
#     save=True
# )

print(f"exp_50: {exp_50}")

In [ ]:
"""creates var exp graphs (from folder!)"""
# import os
# import numpy as np
# import matplotlib.pyplot as plt
# from astropy.io import fits

def var_exp_graphs_from_folder(
    path_folder,
    e_per_adu,
    min_percent=0.05,
    max_percent=0.95,
    save=False
):
    files = sorted(f for f in os.listdir(path_folder) if f.lower().endswith('.fits'))
    if not files:
        raise RuntimeError(f"No FITS files in {path_folder}")

    exp_times = []
    mean_signal = []
    std_signal = []
    for fn in files:
        hdr = fits.getheader(os.path.join(path_folder, fn))
        data = fits.getdata(os.path.join(path_folder, fn)).astype(float)
        exp = hdr.get('EXPTIME')
        if exp is None:
            raise KeyError(f"EXPTIME missing in {fn}")
        exp_times.append(exp)
        mean_signal.append(data.mean())
        std_signal.append(data.std())

    exp_times   = np.array(exp_times)
    mean_signal = np.array(mean_signal)
    std_signal  = np.array(std_signal)
    exp_ms      = exp_times * 1e3

    # --- Compute full-well ---
    hdr0 = fits.getheader(os.path.join(path_folder, files[0]))
    bitpix = hdr0.get('BITPIX', 16)
    bit_max = 2**bitpix - 1

    # --- Find linear region indices ---
    low_idx  = np.argmax(mean_signal >= min_percent * bit_max)
    high_idx = np.argmax(mean_signal >= max_percent * bit_max)

    # --- Fit linear region ---
    m, b = np.polyfit(
        exp_ms[low_idx:high_idx+1],
        mean_signal[low_idx:high_idx+1],
        1
    )

    # --- Prepare figure ---
    fig, (ax1, ax2) = plt.subplots(1,2, figsize=(12,4))

    # Panel a) linear-scale
    ax1.errorbar(exp_ms, mean_signal, yerr=std_signal,
                 fmt='o', capsize=3, label='Measured')
    x_lin = np.linspace(exp_ms[0], exp_ms[high_idx], 200)
    ax1.plot(x_lin, m*x_lin + b, 'r--', label='Linear fit')
    ax1.plot(x_lin, m*x_lin, 'b-.', label='Ideal')
    # saturation line
    ax1.axhline(bit_max, ls=':', color='gray')
    ax1.text(0.99, bit_max, 'Saturation', ha='right', va='bottom', transform=ax1.get_yaxis_transform())
    # region bounds
    xp_low  = exp_ms[low_idx]
    xp_high = exp_ms[high_idx]
    ax1.axvline(xp_low,  ls=':', color='gray')
    ax1.text(xp_low, 0.01, f'{min_percent*100:.0f}%', ha='center', va='bottom', transform=ax1.get_xaxis_transform())
    ax1.axvline(xp_high, ls=':', color='gray')
    ax1.text(xp_high,0.01, f'{max_percent*100:.0f}%', ha='center', va='bottom', transform=ax1.get_xaxis_transform())
    ax1.set_xlabel('Exposure (ms)')
    ax1.set_ylabel('Mean signal (ADU)')
    ax1.set_title('a) Full Response')
    ax1.legend(fontsize='small')
    ax1.grid(True, ls='--', lw=0.5)
    # secondary axis
    ax1b = ax1.twinx()
    lo, hi = ax1.get_ylim()
    ax1b.set_ylim(lo*e_per_adu, hi*e_per_adu)
    ax1b.set_ylabel('Mean signal (e-)')

    # Panel b) log-log
    ax2.errorbar(exp_ms, mean_signal, yerr=std_signal,
                 fmt='o', capsize=3, label='Measured')
    ax2.plot(x_lin, m*x_lin + b, 'r--', label='Linear fit')
    ax2.plot(x_lin, m*x_lin, 'b-.', label='Ideal')
    ax2.set_xscale('log')
    ax2.set_yscale('log')
    ax2.axhline(bit_max, ls=':', color='gray')
    ax2.text(0.99, bit_max, 'Saturation', ha='right', va='bottom', transform=ax2.get_yaxis_transform())
    ax2.axvline(xp_low,  ls=':', color='gray')
    ax2.text(xp_low,0.01, f'{min_percent*100:.0f}%', ha='center', va='bottom', transform=ax2.get_xaxis_transform())
    ax2.axvline(xp_high, ls=':', color='gray')
    ax2.text(xp_high,0.01, f'{max_percent*100:.0f}%', ha='center', va='bottom', transform=ax2.get_xaxis_transform())
    ax2.set_xlabel('Exposure (ms)')
    ax2.set_ylabel('Mean signal (ADU)')
    ax2.set_title('b) Full Response, log scale')
    ax2.legend(fontsize='small')
    ax2.grid(True, ls='--', lw=0.5)
    # secondary
    ax2b = ax2.twinx()
    ax2b.set_xscale('log'); ax2b.set_yscale('log')
    ax2b.set_ylabel('Mean signal (e-)')
    lo, hi = ax2.get_ylim()
    ax2b.set_ylim(lo*e_per_adu, hi*e_per_adu)

    plt.tight_layout()
    if save:
        fig.savefig('var_exp_graphs.png', dpi=300)
    plt.show()


In [ ]:
var_exp_graphs_from_folder(
    path_folder="C:/Users/Jonah/fits_imgs/var_exp",
    save=True,
    e_per_adu=.01,
    min_percent=0.01,
    max_percent=0.95,
)

In [ ]:
"""var filter test"""
# Picoammeter worker
def picoam_worker_filters(ser, stop_event, readings_list, times_list, sample_interval=0.004):
    while not stop_event.is_set():
        time.sleep(sample_interval)
        raw = query(ser, 'READ?')
        readings_list.append(float(raw))
        times_list.append(time.time())
    ser.close()

def camera_with_filters_hama(
    iDevice,
    exposure_s,
    nd_positions,
    handle,
    frames_per_filter=5,
    target_shape=(1000,1000),
    save_path=None,
    full=False
):

    pythoncom.CoInitializeEx(pythoncom.COINIT_MULTITHREADED)
    if not Dcamapi.init():
        raise RuntimeError(f"DCAM‑API init failed: {Dcamapi.lasterr()}")

    try:
        cam = Dcam(iDevice)
        if not cam.dev_open():
            raise RuntimeError(f"dev_open failed: {cam.lasterr()}")
        # if not cam.buf_alloc(1):
        #     raise RuntimeError(f"buf_alloc failed: {cam.lasterr()}")

        # set mode to ultra quite

        # Grab full sensor dims
        full_h = int(cam.prop_getvalue(DCAM_IDPROP.IMAGE_HEIGHT))
        full_w = int(cam.prop_getvalue(DCAM_IDPROP.IMAGE_WIDTH))
        if full:
            ty, tx = full_h, full_w
        else:
            ty, tx = target_shape

        cam.prop_setvalue(DCAM_IDPROP.SUBARRAYMODE, 1)
        print("submode", cam.prop_getvalue(DCAM_IDPROP.SUBARRAYMODE))
        cam.prop_setvalue(DCAM_IDPROP.SUBARRAYHPOS,  0)
        cam.prop_setvalue(DCAM_IDPROP.SUBARRAYVPOS,  0)
        cam.prop_setvalue(DCAM_IDPROP.SUBARRAYHSIZE, full_w)
        cam.prop_setvalue(DCAM_IDPROP.SUBARRAYVSIZE, full_h)

        dprop = DCAM_IDPROP.READOUTSPEED
        cam.prop_setvalue(dprop, 1)
        print("Set", 'SOMEPROP', "→", cam.prop_getvalue(dprop))

        cam.buf_alloc(1)

        num_filters = len(nd_positions)
        # Storage
        imgs  = np.zeros((num_filters, frames_per_filter, ty, tx), dtype=np.uint16)
        times = np.zeros((num_filters, frames_per_filter))

        # Loop filters
        for i, pos in enumerate(nd_positions):
            print("starting filter", pos)
            # move wheel
            set_position(handle, pos)
            time.sleep(0.5)

            # set exposure
            cam.prop_setvalue(DCAM_IDPROP.EXPOSURETIME, float(exposure_s))

            for j in range(frames_per_filter):
                if not cam.cap_snapshot():
                    raise RuntimeError(f"cap_snapshot failed: {cam.lasterr()}")
                timeout_ms = int(exposure_s*1000)+2000
                while True:
                    if cam.wait_capevent_frameready(timeout_ms):
                        break
                    if cam.lasterr().is_timeout():
                        continue
                    raise RuntimeError(f"wait_frameready failed: {cam.lasterr()}")
                data = cam.buf_getlastframedata()
                if data.ndim == 1:
                    data = data.reshape(full_h, full_w)

                if full:
                    crop = data.astype(np.uint16)
                else:
                    sy = (full_h-ty)//2
                    sx = (full_w-tx)//2
                    crop = data[sy:sy+ty, sx:sx+tx].astype(np.uint16)
                imgs[i,j]  = crop
                times[i,j] = time.time()

            # save FITS cube per filter
            if save_path:
                os.makedirs(save_path, exist_ok=True)
                cube = imgs[i]
                hdu = fits.PrimaryHDU(cube)
                hdr = hdu.header
                hdr['DATE']    = time.strftime('%Y-%m-%dT%H:%M:%S', time.gmtime())
                hdr['DATE-OBS']    = time.strftime('%Y-%m-%dT%H:%M:%S', time.gmtime())
                hdr['EXPTIME'] = cam.prop_getvalue(DCAM_IDPROP.EXPOSURETIME)
                hdr['TEMP']    = cam.prop_getvalue(DCAM_IDPROP.SENSORTEMPERATURE)
                hdr['MODE']    = int(cam.prop_getvalue(DCAM_IDPROP.READOUTSPEED))
                hdr['GAIN']    = cam.prop_getvalue(DCAM_IDPROP.CONTRASTGAIN)
                hdr['SENSNAME']= cam.dev_getstring(DCAM_IDSTR.MODEL)
                fname = os.path.join(save_path, f"nd_{pos}_cube.fits")
                hdu.writeto(fname, overwrite=True)

        # cleanup
        cam.buf_release()
        cam.dev_close()

    finally:
        Dcamapi.uninit()
        pythoncom.CoUninitialize()
        set_position(handle, 1)

    return imgs, times

def collect_var_filters_hama(
    iDevice=0,
    exposure_s=0.063728,
    nd_positions=[1,2,3,4,5,6],
    handle=None,
    frames_per_filter=5,
    target_shape=(1000,1000),
    com_port='COM5',
    save_path=None,
    full=False
):
    # start pico thread
    ser = serial.Serial(port=com_port, baudrate=9600, timeout=2)
    init_cmds(ser)
    stop_evt = threading.Event()
    pr, pt = [], []
    t = threading.Thread(target=picoam_worker_filters, args=(ser, stop_evt, pr, pt))
    t.start()

    # camera capture
    imgs, times = camera_with_filters_hama(
        iDevice=iDevice,
        exposure_s=exposure_s,
        nd_positions=nd_positions,
        handle=handle,
        frames_per_filter=frames_per_filter,
        target_shape=target_shape,
        save_path=save_path,
        full=full
    )

    # stop picoammeter
    stop_evt.set()
    t.join()

    # build results
    num_f = len(nd_positions)
    # per‑filter mean ADU
    signals_f = imgs.mean(axis=(2,3)).mean(axis=1)
    # mean current per filter
    currents = np.array(pr)
    # interpolate onto each frame time
    currents_interp = np.interp(times.flatten(), np.array(pt), currents)
    I_f = currents_interp.reshape(num_f, frames_per_filter).mean(axis=1)
    # total collected charge per filter
    Q_filt = I_f * exposure_s

    # save per‐frame CSV
    if save_path:
        os.makedirs(save_path, exist_ok=True)
        rows = []
        flat = currents_interp
        for i,pos in enumerate(nd_positions):
            for j in range(frames_per_filter):
                idx = i*frames_per_filter + j
                arr = imgs[i,j]
                rows.append({
                    'filter': pos,
                    'frame':  j,
                    'mean_ADU': arr.mean(),
                    'std_ADU':  arr.std()
                })
        df = pd.DataFrame(rows)
        df.to_csv(os.path.join(save_path,'filter_frame_data.csv'), index=False)
    close_wheels()
    return signals_f, Q_filt

In [ ]:
# ensure your wheel handle is initialized:
handle6 = init_wheels()

signals_f, Q_filt = collect_var_filters_hama(
    iDevice=0,
    exposure_s=4.2,
    nd_positions=[1,2,3,4,5,6],
    handle=handle6,
    frames_per_filter=25,
    target_shape=(1000,1000),
    full=True,
    com_port="COM5",
    save_path="C:/Users/Jonah/fits_imgs"
)

# init_wheels()
# close_wheels()

In [ ]:
def collect_bias_frames_hama(
    iDevice=0,
    exp_low=None,
    num_frames=10,
    img_shape=(1000,1000),
    save_path=None,
    full=False,
    big_file=False,
        num_exp=1,
        index=1
):

    if exp_low is None:
        raise ValueError("exp_low must be specified")

    # prompt for cover
    # cmd = ''
    # while cmd.lower() != 'y':
    #     cmd = input("Is the cover ATTACHED? (y/n) ").strip()

    # capture bias stack via take_ramp_hamamatsu
    exp_times, img_array, img_times = take_ramp_hamamatsu(
        iDevice=iDevice,
        exp_low=exp_low,
        exp_high=exp_low,
        num_exp=num_exp,
        num_frames=num_frames,
        img_shape=img_shape,
        save_path=save_path,
        full=full,
        big_file=big_file
    )
    # print("img array", img_array)
    # img_array shape = (1, num_frames, H, W) --> squeeze to (num_frames, H, W)
    bias_stack = img_array.squeeze(axis=0)
    # print("bias_stack", bias_stack)
    # compute per‐frame means and overall bias
    bias_means = bias_stack.mean(axis=(1,2))     # length=num_frames
    bias_level = float(bias_means.mean())

    # write FITS cube
    if save_path:
        os.makedirs(save_path, exist_ok=True)
        cube_path = os.path.join(save_path, f"bias_stack{index}.fits")
        # Re‑open camera briefly to populate headers
        pythoncom.CoInitializeEx(pythoncom.COINIT_MULTITHREADED)
        if not Dcamapi.init():
            raise RuntimeError("DCAM‑API init failed")

        try:
            dcam = Dcam(iDevice)
            if not dcam.dev_open():
                raise RuntimeError("dev_open failed")

            # Build the header
            hdu = fits.PrimaryHDU(bias_stack.astype(np.uint16))
            hdr = hdu.header
            hdr['DATE'] = time.strftime('%Y-%m-%dT%H:%M:%S', time.gmtime())
            hdr['DATE-OBS'] = time.strftime('%Y-%m-%dT%H:%M:%S', time.gmtime())
            hdr['EXPTIME'] = dcam.prop_getvalue(DCAM_IDPROP.EXPOSURETIME)
            hdr['TEMP'] = dcam.prop_getvalue(DCAM_IDPROP.SENSORTEMPERATURE)
            hdr['MODE'] = int(dcam.prop_getvalue(DCAM_IDPROP.READOUTSPEED))
            hdr['GAIN'] = dcam.prop_getvalue(DCAM_IDPROP.CONTRASTGAIN)
            hdr['SENSNAME']= dcam.dev_getstring(DCAM_IDSTR.MODEL)
            hdul = fits.HDUList([hdu])
            hdul.writeto(cube_path, overwrite=True)

            print(f"Bias cube saved to {cube_path}")
            dcam.dev_close()
        finally:
            Dcamapi.uninit()
            pythoncom.CoUninitialize()

    # save per‐frame means
    if save_path:
        csv_path = os.path.join(save_path, f"bias_means{index}.csv")
        df = pd.DataFrame({
            'frame_index': np.arange(num_frames),
            'mean_ADU': bias_means
        })
        df.to_csv(csv_path, index=False)
        print(f"Bias means saved to {csv_path}")

    # prompt to remove cover
    # cmd = ''
    # while cmd.lower() != 'y':
    #     cmd = input("Is the cover REMOVED? (y/n) ").strip()

    return bias_level

In [ ]:
bias_levels = []
# change loop and num frames if you need smaller file sizes
for i in range (1):
    bias_level = collect_bias_frames_hama(
        iDevice=0,
        exp_low=0.0001,
        num_frames=500,
        img_shape=(1000,1000),
        full=True,
        save_path="C:/Users/Jonah/fits_imgs",
        big_file=True,
        index=i
    )
    bias_levels.append(bias_level)
    print("finished loop")
print("bias levels: ", bias_levels)
bias_levels = np.array(bias_levels)
bias_level = np.mean(bias_levels, axis=0)

print("Computed bias level:", bias_level)

In [ ]:
"""if you ran everything in this runtime, this is faster. copied"""
def compare_exp_and_filter(e_per_adu = None, Q_ramp=None, Q_filt=None, signals_f=None, mean_signal=None, bias_level=None, save=False):
    # print(bias_level)
    sig_ramp = mean_signal - bias_level
    sig_filt = signals_f - bias_level
    # sig_ramp[sig_ramp < 0] = 0
    # sig_filt[sig_filt < 0] = 0
    fig, (ax1) = plt.subplots(1, 1, figsize=(8,6))
    ax1.plot(Q_ramp,    sig_ramp,  'o-', color='C0', label='var exposure')
    ax1.plot(Q_filt,    sig_filt, 's--', color='C1', label='var filters')
    ax1.set_xlabel('Total collected charge $Q=I\\times t$ (A·s)')
    ax1.set_ylabel('Mean signal (ADU)')
    ax1.set_title('Camera Response vs. Total Illumination')
    ax1.legend()
    ax1.grid(ls='--', lw=0.5)
    ax1.set_yscale('log')
    ax1.set_xscale('log')
    ax2 = ax1.twinx()
    low, high = ax1.get_ylim()
    ax2.set_ylim(low*e_per_adu, high*e_per_adu)
    ax2.set_ylabel('Mean signal (e-)')
    ax2.set_yscale('log')

    if save:
        path_base = f"exp_vs_filter"
        fig_path = path_base+"_figure.png"
        fig.savefig(fig_path, dpi=300)
        print(f"Figure saved to {fig_path}")

In [ ]:
compare_exp_and_filter(e_per_adu =e_per_adu, Q_ramp=Q_ramp, Q_filt=Q_filt, signals_f=signals_f, mean_signal=mean_signal, bias_level=bias_level, save=True)

In [ ]:
"""Compares var exp and var filter (from folders!)"""
# import os
# import numpy as np
# import matplotlib.pyplot as plt
# from astropy.io import fits

def compare_exp_and_filter_from_folders(
    path_exp_folder,
    path_filter_folder,
    bias_level=0,
    save=False,
    e_per_adu=1
):
    # Helper to load signals and x-values
    def load_data(folder, x_key):
        x_vals = []
        signals = []
        for fname in sorted(os.listdir(folder)):
            if not fname.lower().endswith('.fits'):
                continue
            path = os.path.join(folder, fname)
            with fits.open(path) as hdul:
                data = hdul[0].data.astype(float)
                hdr = hdul[0].header
                mean_adu = data.mean() - bias_level
                x = hdr.get(x_key)
                if x is None:
                    raise KeyError(f"Header key '{x_key}' missing in {fname}")
                x_vals.append(float(x))
                signals.append(mean_adu)
        return np.array(x_vals), np.array(signals)

        # Load variable exposure data (x=EXPTIME)
    Q_ramp, sig_ramp = load_data(path_exp_folder, 'EXPTIME')

    # Load variable filter data: use order to assign OD values
    # assume 6 files in fixed order: 1:no filter,2:ND0.5,3:ND1.0,4:ND2.0,5:ND3.0,6:ND4.0
    fnames = sorted([f for f in os.listdir(path_filter_folder) if f.lower().endswith('.fits')])
    if len(fnames) != 6:
        raise ValueError(f"Expected 6 filter FITS files, found {len(fnames)}")
    od_list = [0.0, 0.5, 1.0, 2.0, 3.0, 4.0]
    Q_filt    = []
    sig_filt  = []
    T_filt    = None

    for od, fname in zip(od_list, fnames):
        with fits.open(os.path.join(path_filter_folder, fname)) as hdul:
            data = hdul[0].data.astype(float)
            hdr  = hdul[0].header

            mean_adu = data.mean() - bias_level
            # get the exposure time used for all filter frames:
            if T_filt is None:
                T_filt = float(hdr['EXPTIME'])

            # compute relative illumination = 10^(-OD)
            illum_rel = 10.0 ** (-od)
            # total “dose” proxy on same scale as T*I_fixed
            Q_point = T_filt * illum_rel

            Q_filt.append(Q_point)
            sig_filt.append(mean_adu)

    Q_filt   = np.array(Q_filt)
    sig_filt = np.array(sig_filt)

    Q_filt = np.array(Q_filt)
    sig_filt = np.array(sig_filt)

    # Mask out nonpositive for log scale
    mask_r = (Q_ramp > 0) & (sig_ramp > 0)
    mask_f = (Q_filt > 0) & (sig_filt > 0)
    if not mask_r.any():
        raise ValueError("No positive points in variable-exposure data for log plot")
    if not mask_f.any():
        raise ValueError("No positive points in variable-filter data for log plot")

    # Apply masks
    Q_ramp = Q_ramp[mask_r];  sig_ramp = sig_ramp[mask_r]
    Q_filt = Q_filt[mask_f];  sig_filt = sig_filt[mask_f]
# for log scale
    mask_r = (Q_ramp > 0) & (sig_ramp > 0)
    mask_f = (Q_filt > 0) & (sig_filt > 0)
    if not mask_r.any():
        raise ValueError("No positive points in variable-exposure data for log plot")
    if not mask_f.any():
        raise ValueError("No positive points in variable-filter data for log plot")

    # Apply masks
    Q_ramp = Q_ramp[mask_r];  sig_ramp = sig_ramp[mask_r]
    Q_filt = Q_filt[mask_f];  sig_filt = sig_filt[mask_f]

    # Create plot
    fig, ax1 = plt.subplots(figsize=(8,6))
    ax1.plot(Q_ramp, sig_ramp, 'o-', color='C0', label='var exposure')
    ax1.plot(Q_filt, sig_filt, 's--', color='C1', label='var filters')
    ax1.set_xlabel('Total collected charge Q = I x t (A*s)')
    ax1.set_ylabel('Mean signal (ADU)')
    ax1.set_xscale('log')
    ax1.set_yscale('log')
    ax1.set_title('Camera Response vs. Illumination')
    ax1.legend()
    ax1.grid(ls='--', lw=0.5)

    # Secondary axis: electrons
    ax2 = ax1.twinx()
    low, high = ax1.get_ylim()
    # Derive e- axis from first exposure file's GAIN header
    low_e, high_e = low * e_per_adu, high * e_per_adu
    # Avoid identical limits
    if low_e >= high_e:
        low_e, high_e = low_e * 0.9, high_e * 1.1
    ax2.set_ylim(low_e, high_e)
    ax2.set_ylabel('Mean signal (e-)')
    ax2.set_yscale('log')

    if save:
        out = 'exp_vs_filter.png'
        fig.savefig(out, dpi=300)
        print(f"Saved figure to {out}")
    plt.show()


In [ ]:
compare_exp_and_filter_from_folders(
    path_exp_folder="C:/Users/Jonah/fits_imgs/var_exp",
    path_filter_folder="C:/Users/Jonah/fits_imgs/var_filt",
    bias_level=bias_level,
    save=True,
    e_per_adu=e_per_adu
)